### <font color='289C4E'>目录</font><a class='anchor' id='top'></a>
- [导入数据](#1)
- [标准化方向数据](#2)
- [合并数据](#3)
- [创建比赛摘要](#4)
- [探索性数据分析](#5)
- [建模](#6)
- [GIF/动画](#7)

# 1. 导入数据 <a class="anchor"  id="1"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [ ]:
# 读取 games.csv 文件
games_data = pd.read_csv('../../Dataset/games.csv')

# 读取 players.csv 文件
players_data = pd.read_csv('../../Dataset/players.csv')

# 读取 plays.csv 文件
plays_data = pd.read_csv('../../Dataset/plays.csv')

# 读取 tackles.csv 文件
tackles_data = pd.read_csv('../../Dataset/tackles.csv')

建立一个函数用于统计表格相关列的值信息

In [ ]:
def generate_summary_table(data):
    summary_table = pd.DataFrame(columns=['Column', 'Data Type', 'Missing Values', 'Missing %', 'Unique Values', 'Min', 'Max', 'Mean', 'Median'])
    
    for column in data.columns:
        data_type = str(data[column].dtype)
        
        missing_values = data[column].isnull().sum()
        missing_percentage = (missing_values / len(data)) * 100
        
        if data[column].dtype == 'object':
            min_value, max_value, mean_value, median_value = '', '', '', ''
        else:
            min_value = data[column].min()
            max_value = data[column].max()
            mean_value = data[column].mean()
            median_value = data[column].median()
        
        unique_values = data[column].nunique()  # 计算唯一值数量
        
        summary_table = summary_table.append({
            'Column': column,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing %': f'{missing_percentage:.2f}%',
            'Unique Values': unique_values,  # 添加唯一值数量
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value
        }, ignore_index=True)
    
    formatted_table = summary_table.style.set_properties(**{'text-align': 'center'})
    
    print("表格信息统计:")
    display(formatted_table)
    
    return summary_table


1. 比赛数据games.csv分析

In [ ]:
games_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

2. 球员数据players.csv分析

In [ ]:
players_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
players_data_summary = generate_summary_table(players_data)

3. 每场比赛数据plays.csv分析

In [ ]:
plays_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

4. 铲球数据tackles.csv分析

In [ ]:
tackles_data.head()

生成表格信息统计图

In [ ]:
# 调用函数并打印生成的表格
tackles_data_summary = generate_summary_table(tackles_data)

# 2. 标准化数据 <a class="anchor"  id="1"></a>

1. games.csv

首先对该表格内容进行数据清洗与可视化

由之前信息可得此表中没有出现数据缺失的情况，因此可以直接处理

得分分布 - 直方图展示主队和客队得分的分布情况

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

In [ ]:
# 创建直方图
fig = px.histogram(games_data, x=["homeFinalScore", "visitorFinalScore"], nbins=20, barmode="overlay",
                   labels={"value": "Score", "variable": "Team"}, title="Score Distribution")
fig.update_layout(xaxis_title="Score", yaxis_title="Frequency")
fig.show()


赛季中的队伍表现 - 折线图显示每支球队在赛季不同周次的得分情况

In [ ]:
# 根据赛季、周次和队伍分组并计算得分总和
team_scores = games_data.groupby(["season", "week", "homeTeamAbbr"])["homeFinalScore"].sum().reset_index()
team_scores = team_scores.rename(columns={"homeTeamAbbr": "Team", "homeFinalScore": "Score"})

# 获取所有队伍的列表
all_teams = team_scores["Team"].unique().tolist()

# 创建交互式折线图
fig = go.Figure()

for team in all_teams:
    team_data = team_scores[team_scores["Team"] == team]
    fig.add_trace(go.Scatter(x=team_data["week"], y=team_data["Score"], mode='lines', name=team))

# 设置布局
fig.update_layout(
    title="Team Performance in Different Weeks",
    xaxis_title="Week",
    yaxis_title="Score",
    legend=dict(orientation="h", y=-0.2),
    margin=dict(l=20, r=20, t=80, b=20)
)

fig.show()

从结果可以看出，并不是所有的队伍每个星期都参赛

赛季中比赛结果统计 - 饼图展示每个赛季主队和客队的胜利次数和比赛结果

In [ ]:
# 创建新列，表示主队和客队的胜利情况
games_data['home_win'] = games_data['homeFinalScore'] > games_data['visitorFinalScore']
games_data['visitor_win'] = games_data['homeFinalScore'] < games_data['visitorFinalScore']

# 统计每个赛季中主队和客队的胜利次数
season_results = games_data.groupby(['season']).agg({
    'home_win': 'sum',
    'visitor_win': 'sum'
}).reset_index()

# 重塑数据以符合饼图格式
season_results_melted = season_results.melt(id_vars='season', var_name='Result', value_name='Wins')

# 显示饼图
fig = px.pie(season_results_melted, values='Wins', names='Result', title='Season Results: Home Team vs Visitor Team Wins',
             hover_data=['Wins'], labels={'Result': 'Game Result'})
fig.show()

看来主场确实有优势啊

接下来进行数据清理的工作

In [ ]:
# 调用函数并打印生成的表格
games_data_summary = generate_summary_table(games_data)

分析结果发现这里面存在着大量的非数值文本，这会使数据很难处理，因此我们要适当的舍弃或者对这些文本进行重新编码。接下来我们看看这些列分别代表了什么数据

这些字段的含义如下：

- **gameId（比赛ID）**：比赛的唯一标识符（数字）
- **season（赛季）**：比赛所属的赛季（数字）
- **week（周次）**：比赛所在的周次（数字）
- **gameDate（比赛日期）**：比赛日期（时间，月/日/年）
- **gameTimeEastern（比赛开始时间）**：比赛开始时间（时间，时:分:秒，东部标准时间）
- **homeTeamAbbr（主队三字母代码）**：主队的三字母代码（文本）
- **visitorTeamAbbr（客队三字母代码）**：客队的三字母代码（文本）
- **homeFinalScore（主队最终得分）**：主队在比赛中的总得分（数字）
- **visitorFinalScore（客队最终得分）**：客队在比赛中的总得分（数字）

这些字段描述了每场比赛的详细信息，包括比赛的标识、赛季、比赛日期和时间、参赛队伍及其得分情况。

* 其中赛季（season）肯定是一个无关值，因为这些数据都属于一个赛季，可以抛弃
* gameTimeEastern可能不太重要，因为是比赛举行的时间，稍后可以用热力图分析一下
* 而对主队客队，我们可以采用编码的方式来对这些队伍进行编码来剔除文本信息。

由于热力图只能处理数值信息，因此我们应先将队名，比赛日期和比赛时间进行编码

In [ ]:
# 获取所有队伍的列表，并按字母顺序排序
teams = sorted(set(games_data['homeTeamAbbr'].unique()) | set(games_data['visitorTeamAbbr'].unique()))

# 创建队伍编码映射
team_encoding = {team: code for code, team in enumerate(teams, 1)}

# 对 'homeTeamAbbr' 和 'visitorTeamAbbr' 进行编码
games_data['homeTeamAbbr'] = games_data['homeTeamAbbr'].map(team_encoding)
games_data['visitorTeamAbbr'] = games_data['visitorTeamAbbr'].map(team_encoding)

# 对 'gameDate' 进行日期编码
games_data['gameDate'] = pd.to_datetime(games_data['gameDate'])
games_data['gameDate'] = (games_data['gameDate'] - games_data['gameDate'].min()).dt.days + 1

# 对 'gameTimeEastern' 进行小时编码
games_data['gameTimeEastern'] = pd.to_datetime(games_data['gameTimeEastern']).dt.hour + \
                                pd.to_datetime(games_data['gameTimeEastern']).dt.minute / 60 + \
                                pd.to_datetime(games_data['gameTimeEastern']).dt.second / 3600

# 输出队名与编码的对应关系
for team, code in team_encoding.items():
    print(f"Team: {team} -> Code: {code}")

# 输出编码后的结果
print(games_data[['season', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'homeFinalScore', 'visitorFinalScore', 'gameDate', 'gameTimeEastern']])

对于队名我采用了一个编码规则，按照字母排列编码
比赛日期也采用了相似的编码，比赛时间编码则是直接转化为小时数，比如20:00:00就编码为20

首先drop掉season，再画出热力图，看看比赛时间会不会对比赛的胜负结果产生影响。

In [ ]:
# 剔除 'season', 'home_win', 'visitor_win' 列
games_df_without_season = games_data.drop(columns=['season', 'home_win', 'visitor_win'])

# 计算相关性矩阵
correlation_matrix = games_df_without_season.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns (Season Column Excluded)',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations
)

fig.show()

发现gameId,week和gameDate强相关，这是因为比赛的时间安排决定了这几项必然强相关，相反，其他数据项的相关性并不大，因此，我们需要进一步分析其相关性。

采用随机森林的方法，目标变量为'homeFinalScore', 'visitorFinalScore'，因为这两列数据是比分数据

In [ ]:
# 定义特征 (X) 和目标变量 (y)
X = games_df_without_season.drop(['homeFinalScore', 'visitorFinalScore'], axis=1)
y = games_df_without_season[['homeFinalScore', 'visitorFinalScore']]

# 划分数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建一个包含 100 个决策树的随机森林回归模型
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# 拟合模型
rf_regressor.fit(X_train, y_train)

# 获取特征的重要性
feature_importances = rf_regressor.feature_importances_

# 创建 DataFrame 来可视化特征重要性
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# 对特征重要性值进行降序排列
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 使用 Plotly Express 创建水平条形图
fig = px.bar(feature_importance_df, x='Importance', y='Feature', orientation='h',
             labels={'Importance': 'Importance', 'Feature': 'Feature'},
             title='Feature Importances')
fig.show()

由上图所示，我们可以放心剔除week，gameDate和gameTimeEastern来减少数据维度，这些数据列与比赛的分数相关性不大，保留gameId，因为可能要用这项数据连接其他表。至于'home_win', 'visitor_win'两列仅仅代表主队胜利还是客队胜利，这一点通过比较得分就能得到，没有必要保留徒增数据维度。

In [ ]:
games_data_cleaned = games_data.drop(columns=['season', 'home_win', 'visitor_win', 'week', 'gameDate', 'gameTimeEastern'])
games_data_cleaned.head()  # 数据清理完成

至此，该表的数据清理完成

2. players.csv

首先对该表格内容进行公制化转换，转换成我们可以处理的数据

In [ ]:
players_data.head()

身高：米  体重：千克  出生日期：转化为年龄

In [ ]:
# 转换身高为厘米
players_data['height'] = players_data['height'].apply(lambda x: int(x.split('-')[0]) * 30.48 + int(x.split('-')[1]) * 2.54)

# 转换体重为公斤
players_data['weight'] = players_data['weight'] * 0.453592

# 计算年龄
players_data['birthDate'] = 2022 - pd.to_datetime(players_data['birthDate']).dt.year

# 输出转换后的结果
print(players_data)

In [ ]:
# 调用函数并打印生成的表格
players_data_summary = generate_summary_table(players_data)

由之前的统计信息可知，此表在生日值上是有缺失的，因此很多人的年龄不知道，并且缺失值达到了28.46%，不能采用中值或者均值进行替换，因此我们不得不舍弃该数据列，尽管在体育上年龄对于球员表现是重要因素。

而且球员姓名也有重名，因此我们不能对球员姓名进行编码，只能使用nflId来标识球员

In [ ]:
# 删除 'birthDate' 列
players_data.drop(['birthDate'], axis=1, inplace=True)

# 输出结果
print(players_data)

接下来我们进行可视化分析

In [ ]:
# 创建直方图统计身高每一厘米的频率
height_counts = players_data['height'].value_counts().sort_index()

fig = go.Figure(data=[go.Bar(x=height_counts.index, y=height_counts.values)])
fig.update_layout(
    title='Height Distribution',
    xaxis_title='Height (cm)',
    yaxis_title='Frequency'
)
fig.show()

In [ ]:
# 创建直方图统计体重
fig_weight = px.histogram(players_data, x='weight', title='Weight Distribution')
fig_weight.update_xaxes(title='Weight')
fig_weight.show()

In [ ]:
# 创建直方图不同位置的球员数量
fig_position = px.histogram(players_data, x='position', title='Player Position Distribution')
fig_position.update_xaxes(title='Position')
fig_position.show()

In [ ]:
# 创建箱线图得到位置与体重的关系
fig_box = px.box(players_data, x='position', y='weight', title='Position vs Weight')
fig_box.update_xaxes(title='Position')
fig_box.update_yaxes(title='Weight')
fig_box.show()

In [ ]:
# 计算各个球员位置的平均身高和体重
avg_height_weight = players_data.groupby('position').agg({'height': 'mean', 'weight': 'mean'}).reset_index()

# 绘制柱状图展示不同位置球员的平均身高和体重
fig_avg = px.bar(avg_height_weight, x='position', y=['height', 'weight'], 
                 barmode='group', title='Average Height and Weight by Position')
fig_avg.update_xaxes(title='Position')
fig_avg.update_yaxes(title='Value')
fig_avg.show()

In [ ]:
# 计算BMI指数
players_data['bmi'] = players_data['weight'] / ((players_data['height'] / 100) ** 2)

# 绘制BMI指数的直方图
fig_bmi = px.histogram(players_data, x='bmi', title='BMI Distribution')
fig_bmi.update_xaxes(title='BMI')
fig_bmi.show()

可视化分析完成后，接下来对其collegeName和position两列进行编码，将姓名drop掉使用nflId标识球员

In [ ]:
# 获取所有学院和职位的列表，并按字母顺序排序
college_names = sorted(players_data['collegeName'].unique())
positions = sorted(players_data['position'].unique())

# 创建学院和职位编码映射
college_encoding = {college: code for code, college in enumerate(college_names, 1)}
position_encoding = {position: code for code, position in enumerate(positions, 1)}

# 对 'collegeName' 和 'position' 进行编码
players_data['collegeName'] = players_data['collegeName'].map(college_encoding)
players_data['position'] = players_data['position'].map(position_encoding)

# 删除 'displayName' 列
players_data.drop(['displayName'], axis=1, inplace=True)

# 输出编码后的结果
print(players_data)

接下来绘制热力图，来分析数据列的关系

In [ ]:
# 计算相关性矩阵
correlation_matrix = players_data.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations
)

fig.show()

分析热力图可以得出相关性较高的数据项是身高体重和bmi，这一方面是因为人的成长决定的，人高和体重必然存在一定的联系，另一方面是因为bmi是由身高体重计算得出的。

我们将这三项drop掉，再观察其热力图分布

In [ ]:
# 删除 'height'、'weight' 和 'bmi' 列
players_data_deleted = players_data.drop(columns=['height', 'weight', 'bmi'])

# 计算相关性矩阵
correlation_matrix = players_data_deleted.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns (height, weight, bmi Excluded)',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations
)

fig.show()

可以看到剩下的数据列都是一些球员的属性名，比如球员打的位置和大学名称，几乎没有任何相关性，因此该表中的数据几乎不用清洗掉这些列的内容，因为这些列还要负责后续表格的连接工作

In [ ]:
players_data_cleaned = players_data
print(players_data_cleaned)

3. plays.csv

首先对该表格内容进行数据清洗与可视化

In [ ]:
plays_data.head()

In [ ]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

可以看到表格当中存在很多缺失值，我们用以下代码进行直观统计

In [ ]:
# 找出有缺失值的列
columns_with_missing_values = plays_data.columns[plays_data.isnull().any()]
missing_values_info = []

# 遍历每一列，统计缺失值的数量、百分比和数据类型
for column in columns_with_missing_values:
    missing_count = plays_data[column].isnull().sum()
    missing_percentage = (missing_count / len(plays_data)) * 100
    data_type = plays_data[column].dtype
    missing_values_info.append((column, missing_percentage, missing_count, data_type))

# 将结果转换为 DataFrame 并按数据类型排序
missing_values_df = pd.DataFrame(missing_values_info, columns=['Column Name', 'Missing Percentage', 'Missing Count', 'Data Type'])
missing_values_df = missing_values_df.sort_values(by='Data Type')

# 打印结果
print(missing_values_df)

按照数据类型排序可以发现其中defendersInTheBox，passProbability，expectedPointsAdded，yardlineSide，offenseFormation缺失值较少，剩余数据项缺失值很多。
passLength，passResult，penaltyYards的缺失是因为可能在此期间没有发生传球活动或者球员没有犯规，foulName的缺失则是因为球员可能没有犯规

首先drop掉与犯规的数据

In [ ]:
# 确定要删除的与犯规相关的列名列表
columns_to_drop = ['penaltyYards', 'foulNFLId1', 'foulNFLId2', 'foulName1', 'foulName2']

# 删除这些列
plays_data.drop(columns=columns_to_drop, inplace=True)

# 打印删除列后的数据集
plays_data.head()

剩下的数据维度依旧有30维，我们继续清洗

通过观察我们发现
* ballCarrierId和ballCarrierDisplayName就是球员的对应关系，并且与之前的players.csv表中的nflId能够对应起来，因此我们将ballCarrierDisplayName drop掉，将ballCarrierId改为nflId
* playDescription是长文本信息表示了场上发生的具体事件，难以量化统计，drop掉
* possessionTeam和defensiveTeam可以采用与之前相同的编码方式按照字母顺序从1开始将队名编码，方便量化处理	
* yardlineSide存在缺失值，因此将其drop掉
* gameClock是比赛时间发生的时间（24小时制），由于不同场次的比赛与不同时间点发生（有的是早上开赛有的是晚上开赛），因此该项难以量化统计。也要drop掉
* passResult和passLength存在巨量空值，难以统计，也要drop掉
* playNullifiedByPenalty是一个跟犯规相关的数据，也要drop掉
* offenseFormation缺失值少，存在缺失有可能是根本没有阵型，可以采取数值编码的形式，没有阵型为0，剩下的阵型按照字母顺序从1开始编码
* defendersInTheBox出现少量空值代表在当前时候根本没有防守球员靠近box，可以将空值置0
* passProbability出现少量空值，也可以采用defendersInTheBox类似的做法
* expectedPointsAdded存在一个空值，由于表格庞大，我们可以采用该数据列的中位数进行填充

分析完表格后就可以数据可视化了

不同节次中比赛的分布情况

In [ ]:
# 创建直方图
fig = px.histogram(plays_data, x="quarter", title="Distribution of Games by Quarters")
fig.update_xaxes(title_text="Quarter")
fig.update_yaxes(title_text="Number of Games")
fig.show()

进攻次数在比赛的分布情况

In [ ]:
# 获取不同down的唯一整数值
down_values = sorted(plays_data['down'].unique())

# 创建直方图
fig = px.histogram(plays_data, x='down', title='Distribution of Plays by Down',
                   labels={'down': 'Down', 'count': 'Number of Plays'})

# 自定义横坐标轴的刻度值和标签
fig.update_layout(xaxis=dict(tickvals=down_values, ticktext=[str(val) for val in down_values]))

# 显示图形
fig.show()

前进码数（yards to go）的分布情况

In [ ]:
# 使用 Plotly 绘制前进码数的分布情况
fig = px.histogram(plays_data, x='yardsToGo', title='Distribution of Yards to Go')
fig.update_xaxes(title='Yards to Go')  # 设置 x 轴标题
fig.update_yaxes(title='Frequency')    # 设置 y 轴标题
fig.show()

第三次进攻且距离终点很短（3rd and short）或者第一次进攻但距离终点很远（1st and long）。

基于持球队伍的比赛分布情况possessionTeam

In [ ]:
# 使用value_counts()计算每个持球队伍的比赛数量
possession_team_count = plays_data['possessionTeam'].value_counts()

# 创建基于持球队伍比赛分布情况的柱状图
fig = px.bar(possession_team_count, x=possession_team_count.index, y=possession_team_count.values,
             labels={'x': '持球队伍', 'y': '比赛数量'},
             title='基于持球队伍的比赛分布情况')

fig.update_layout(xaxis={'categoryorder': 'total descending'})  # 按比赛数量降序排列持球队伍

fig.show()

预期分值增加（Expected Points Added，EPA）的分布情况

In [ ]:
fig = px.histogram(plays_data, x='expectedPointsAdded', 
                   title='Distribution of Expected Points Added (EPA)',
                   labels={'expectedPointsAdded': 'Expected Points Added (EPA)'})

fig.update_layout(xaxis_title='Expected Points Added (EPA)',
                  yaxis_title='Count')

fig.show()

EPA衡量了一次比赛对球队预期得分的影响。它有助于评估不同比赛的效果。

可视化完成之后我们对表格信息进行处理

删除处理

In [ ]:
plays_data = plays_data.drop(columns=['ballCarrierDisplayName', 'playDescription', 'yardlineSide', 'gameClock',
                                      'passResult', 'passLength', 'playNullifiedByPenalty'])
plays_data.head()

列名替换

In [ ]:
# 替换列名 'ballCarrierId' 为 'nflId'
plays_data.rename(columns={'ballCarrierId': 'nflId'}, inplace=True)

表格编码
* 'possessionTeam' 和 'defensiveTeam'

In [ ]:
# 创建一个包含 'possessionTeam' 和 'defensiveTeam' 列独特值的列表
unique_teams = sorted(list(set(plays_data['possessionTeam']) | set(plays_data['defensiveTeam'])))

# 创建一个从 1 开始的新编码字典
team_encoding = {team: code for code, team in enumerate(unique_teams, start=1)}

# 使用字典映射将 'possessionTeam' 和 'defensiveTeam' 列的值进行重新编码
plays_data['possessionTeam_encoded'] = plays_data['possessionTeam'].map(team_encoding)
plays_data['defensiveTeam_encoded'] = plays_data['defensiveTeam'].map(team_encoding)

# 将编码值填充回原列
plays_data['possessionTeam'] = plays_data['possessionTeam_encoded']
plays_data['defensiveTeam'] = plays_data['defensiveTeam_encoded']

plays_data.head()

* 'offenseFormation'

In [ ]:
# 将'offenseFormation'列中的NA值替换为0
plays_data['offenseFormation'].fillna(0, inplace=True)

# 排除填充后的整数0，对非零字符串值进行编码
unique_formations = sorted(filter(lambda x: x != 0, plays_data['offenseFormation'].unique()))
encoding_dict = {val: idx + 1 for idx, val in enumerate(unique_formations)}

# 将编码后的值填充回'offenseFormation'列
plays_data['offenseFormation'] = plays_data['offenseFormation'].map(lambda x: encoding_dict.get(x, 0)).astype(int)

# 显示编码结果与原始值的对应关系
reverse_encoding_dict = {v: k for k, v in encoding_dict.items()}
print("编码结果与原始值的对应关系:")
for code, value in sorted(reverse_encoding_dict.items()):
    print(f"{code} -> '{value}'")

defendersInTheBox与 passProbability列的空值置0

In [ ]:
# 将'defendersInTheBox'列和'passProbability'列中的NA值替换为数值0
plays_data['defendersInTheBox'].fillna(0, inplace=True)
plays_data['passProbability'].fillna(0, inplace=True)

中位数填充

In [ ]:
# 计算'expectedPointsAdded'列的中位数
epa_median = plays_data['expectedPointsAdded'].median()

# 将'expectedPointsAdded'列的空值使用中位数填充
plays_data['expectedPointsAdded'].fillna(epa_median, inplace=True)

展示清洗后的数据

In [ ]:
plays_data.head

In [ ]:
plays_data_summary = generate_summary_table(plays_data)

In [ ]:
# 计算相关性矩阵
correlation_matrix = plays_data.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations,
    width=1200,  # 设置宽度为800像素
    height=1200  # 设置高度为600像素 
)

fig.show()

这张图太大了，很难找出相关性，我们把相关性阈值调到0.2，找到相关性高的列

In [ ]:
# 计算相关性矩阵
correlation_matrix = plays_data.corr()

# 找到相关性绝对值大于0.5的列
high_correlation_cols = correlation_matrix[
    (correlation_matrix.abs() > 0.2) & (correlation_matrix != 1)
].stack().reset_index()

high_correlation_cols.columns = ['Column1', 'Column2', 'Correlation']

# 根据高相关性的列重新构建相关性矩阵
filtered_matrix = pd.pivot_table(high_correlation_cols, values='Correlation', 
                                 index='Column1', columns='Column2').fillna(0)

# 创建热力图
fig = px.imshow(filtered_matrix,
                labels=dict(color="Correlation"),
                x=filtered_matrix.index,
                y=filtered_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(filtered_matrix.index):
    for j, col in enumerate(filtered_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{filtered_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

fig.update_layout(
    title='Correlation Heatmap of Columns (|Correlation| > 0.2)',
    xaxis_title='Columns',
    yaxis_title='Columns',
    annotations=annotations,
    width=1000,  # 设置宽度为800像素
    height=800  # 设置高度为800像素
)

fig.show()

In [ ]:
# 打印出高相关性的列名
high_correlation_columns = high_correlation_cols[['Column1', 'Column2']]
high_correlation_columns = high_correlation_columns.stack().unique()
print("高相关性绝对值大于0.2的列名：")
for col in high_correlation_columns:
    print(col)


- **playId**: 每个球场行为（play）的唯一标识符。
- **quarter**: 球赛的季节，表示比赛处于第几节。
- **preSnapHomeScore**: 此次行为（play）之前的主队得分。
- **preSnapVisitorScore**: 此次行为（play）之前的客队得分。
- **down**: 当前球场行为中所处的第几次进攻。
- **yardsToGo**: 到达下一个进攻阵地线所需的码数。
- **passProbability**: 此次进攻行为（play）为传球的概率。
- **defendersInTheBox**: 防守方在中锋（center）和进攻线（offensive line）之间的防守球员数量。
- **expectedPoints**: 此次球场行为（play）所带来的预期积分。
- **preSnapHomeTeamWinProbability**: 此次行为（play）之前主队获胜的概率。
- **preSnapVisitorTeamWinProbability**: 此次行为（play）之前客队获胜的概率。
- **prePenaltyPlayResult**: 在处罚（penalty）之前的球场行为结果。
- **playResult**: 当前球场行为的结果。
- **expectedPointsAdded**: 此次球场行为（play）所带来的预期积分增加值，即根据历史数据和场上情况，此次行为可能带来的积分增加期望值。
- **homeTeamWinProbabilityAdded**: 此次球场行为（play）之后，主队获胜概率的变化。
- **visitorTeamWinProbabilityAdded**: 此次球场行为（play）之后，客队获胜概率的变化。

以上列的相关性高，代表对比赛结果影响较大。

此外我们还要保留gameId,playId,nflID用来连接表

In [ ]:
# 列名列表，保留的列
columns_to_keep = [
    'gameId', 'playId', 'nflId', 'quarter', 'preSnapHomeScore', 'preSnapVisitorScore', 'down',
    'yardsToGo', 'passProbability', 'defendersInTheBox', 'expectedPoints',
    'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
    'prePenaltyPlayResult', 'playResult', 'expectedPointsAdded',
    'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 
]

# 删除除了指定列之外的所有列
plays_data_cleaned = plays_data[columns_to_keep]
plays_data_cleaned.head()

In [ ]:
# 调用函数并打印生成的表格
plays_data_summary = generate_summary_table(plays_data)

没有问题，下一张表走起

4. tackles.csv

首先对该表格内容进行数据清洗与可视化

In [ ]:
tackles_data.head()

In [ ]:
# 调用函数并打印生成的表格
tackles_data_summary = generate_summary_table(tackles_data)

可以看到该表全为数值项，并且没有任何缺失数据，大大减轻了笔者的工作！！！
直接进行可视化分析！！！

铲球（tackles）的分布情况

In [ ]:
# 统计铲球是否成功的次数
tackle_success_counts = tackles_data['tackle'].value_counts()

# 创建条形统计图
fig = px.bar(x=tackle_success_counts.index, y=tackle_success_counts.values,
             labels={'x': 'Tackle Success', 'y': 'Count'},
             title='Distribution of Tackle Success',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 显示图表
fig.show()

助攻（assists）的分布情况

In [ ]:
# 统计助攻是否成功的次数
assist_counts = tackles_data['assist'].value_counts()

# 创建条形统计图
fig = px.bar(x=assist_counts.index, y=assist_counts.values,
             labels={'x': 'Assist Success', 'y': 'Count'},
             title='Distribution of Assist Success',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 显示图表
fig.show()

强制性失误（forced fumbles）的分布情况

In [ ]:
# 统计强制性失误的次数
forced_fumbles_counts = tackles_data['forcedFumble'].value_counts()

# 创建条形统计图
fig = px.bar(x=forced_fumbles_counts.index, y=forced_fumbles_counts.values,
             labels={'x': 'Forced Fumble', 'y': 'Count'},
             title='Distribution of Forced Fumbles',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 显示图表
fig.show()

Pro Football Focus（PFF）失误铲球（missed tackles）的分布情况

In [ ]:
# 统计 PFF 失误铲球的次数
missed_tackles_counts = tackles_data['pff_missedTackle'].value_counts()

# 创建条形统计图
fig = px.bar(x=missed_tackles_counts.index, y=missed_tackles_counts.values,
             labels={'x': 'PFF Missed Tackle', 'y': 'Count'},
             title='Distribution of PFF Missed Tackles',
             category_orders={'x': [0, 1]}
            )

# 设置 x 轴标签
fig.update_xaxes(type='category')

# 显示图表
fig.show()

直接绘制热力图进行分析

In [ ]:
# 计算相关性矩阵
correlation_matrix = tackles_data.corr()

# 创建热力图
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.index,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis')

# 在热力图中标注数值
annotations = []
for i, row in enumerate(correlation_matrix.index):
    for j, col in enumerate(correlation_matrix.columns):
        annotations.append(
            dict(
                x=col,
                y=row,
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
            )
        )

# 更新布局
fig.update_layout(
    title='Correlation Heatmap of Features',
    xaxis_title='Features',
    yaxis_title='Features',
    annotations=annotations,
    width=800,  # 设置宽度为800像素
    height=600  # 设置高度为600像素
)

# 显示图表
fig.show()

从以上图中分析出tackle和assist有很强的负相关性

pff_missedTackle和tackle有很强的负相关性

接下来处理最难处理的表，week表

第一步读取表格并合并

In [ ]:
week1_data = pd.read_csv('../../Dataset/tracking_week_1.csv')
week2_data = pd.read_csv('../../Dataset/tracking_week_2.csv')
week3_data = pd.read_csv('../../Dataset/tracking_week_3.csv')
week4_data = pd.read_csv('../../Dataset/tracking_week_4.csv')
week5_data = pd.read_csv('../../Dataset/tracking_week_5.csv')
week6_data = pd.read_csv('../../Dataset/tracking_week_6.csv')
week7_data = pd.read_csv('../../Dataset/tracking_week_7.csv')
week8_data = pd.read_csv('../../Dataset/tracking_week_8.csv')
week9_data = pd.read_csv('../../Dataset/tracking_week_9.csv')